<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/CatOrDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob 
import os.path as osp 
import random 
import numpy as np 
import json 
from PIL import Image 
from tqdm import tqdm 
import matplotlib.pyplot as plt 
%matplotlib inline 

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.utils.data as data 
import torchvision 
from torchvision import models , transforms

In [0]:
class ImageTransform():


    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        
        return self.data_transform[phase](img)

In [21]:
def make_datapath_list(phase='train'):
  rootpath = '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/'
  target_path = osp.join(rootpath+phase+'/**/*.jpg')
  print(target_path)
  
  path_list = []
  
  #globを利用してサブディレクトリまでパスを取得する
  for path in glob.glob(target_path):
    path_list.append(path)
  
  return path_list

#実行
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

train_list

/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/**/*.jpg
/content/drive/My Drive/Cat_and_Dog/dog_and_cat/val/**/*.jpg


['/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10148422883.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10427257456.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10527416296.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10621815755.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10190610263.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10190618923.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10157877334.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10903921164.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/10672340076.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/1064735160.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/11092051365.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/11054709683.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/dog/11142

In [43]:
size = 224 
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)

#Datasetの作成

class DogCatDataset(data.Dataset):
  """
  犬猫の画像のDatasetクラス.PytorchのDatasetクラスを継承
  """
  def __init__(self,file_list,transform=None,phase='train'):
    self.file_list = file_list
    self.transform = transform 
    self.phase = phase
    
  def __len__(self):
    """画像の枚数を返す"""
    return len(self.file_list)
  
  def __getitem__(self,index):
    '''
    前処理をした画像のTensoor形式のデータとラベルを取得
    '''
    img_path = self.file_list[index]
    img = Image.open(img_path)
    
    #画像の前処理を実施
    img_transformed = self.transform(
        img,self.phase
    ) #torch.Size([3,224,224])
    
    #画像のラベルをファイル名から抜き出す
    if self.phase == 'train':
      label = img_path[54:57] #train/の後の　catかdogを入れたい
    elif self.phase == 'val':
      label = img_path[52:55] #val/の後の　catかdogを入れたい
    
    #ラベルを数値に変換
    if label == 'cat':
      label = 0
    elif label == 'dog':
      label = 1
      
    return img_transformed, label
  
 #実行
train_dataset = DogCatDataset(
      file_list = train_list,transform=ImageTransform(size,mean,std),phase='train'
    )
val_dataset = DogCatDataset(
      file_list = test_list,transform=ImageTransform(size,mean,std),phase='val'
    )
    
#indexの確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])
print(len('/content/drive/My Drive/Cat_and_Dog/dog_and_cat/train/'))
print(len('/content/drive/My Drive/Cat_and_Dog/dog_and_cat/val/'))

torch.Size([3, 224, 224])
1
54
52


In [44]:
# ミニバッチのサイズを指定
batch_size = 32

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 動作確認
batch_iterator = iter(dataloaders_dict["train"])  # イテレータに変換
inputs, labels = next(
    batch_iterator)  # 1番目の要素を取り出す
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 1])


In [29]:
# 学習済みのVGG-16モデルをロード
# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットを犬と猫の2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:11<00:00, 49332468.44it/s]


ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [0]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [31]:

# 転移学習で学習させるパラメータを、変数params_to_updateに格納する
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

# params_to_updateの中身を確認
print("-----------")
print(params_to_update)

classifier.6.weight
classifier.6.bias
-----------
[Parameter containing:
tensor([[-1.4032e-02, -1.4744e-02,  7.4798e-03,  ...,  8.2434e-03,
          2.6599e-06,  1.3044e-02],
        [-4.2980e-03,  9.9745e-03, -7.3846e-03,  ..., -4.0210e-03,
         -5.7769e-03,  1.0164e-02]], requires_grad=True), Parameter containing:
tensor([-0.0092, -0.0044], requires_grad=True)]


In [0]:

# 最適化手法の設定
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

In [41]:
len(dataloaders_dict['val'].dataset)

0

In [0]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測
                    
  
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イタレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)  
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

In [40]:
# 学習・検証を実行する
from __future__ import division
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


0it [00:00, ?it/s]

Epoch 1/2
-------------


ZeroDivisionError: ignored